# Chapter 18: Sensitivity Analysis for the Average Causal Effect with Unmeasured Confounding

In [1]:
import numpy as np
import pandas as pd
from utils import *

np.random.seed(42)
%load_ext autoreload
%autoreload 1

%load_ext watermark
%watermark --iversions

graphviz: 0.20.1
scipy   : 1.13.0
pandas  : 2.2.2
numpy   : 1.26.4



In [2]:
from sklearn.linear_model import LogisticRegression, LinearRegression


def OS_est(
    z,
    y,
    x,
    omod=LinearRegression(),
    pmod=LogisticRegression(C=1e4),
    lb=0,
    ub=1,
    e1=1,
    e0=1,
):
    pscore = pmod.fit(x, z).predict_proba(x)[:, 1]
    pscore = np.clip(pscore, lb, ub)
    # fitted potential outcomes
    outcome1 = omod.fit(x[z == 1, :], y[z == 1]).predict(x)
    outcome0 = omod.fit(x[z == 0, :], y[z == 0]).predict(x)

    ## outcome regression estimator
    ace_reg = (
        np.mean(z * y)
        + np.mean((1 - z) * outcome1 / e1)
        - np.mean(z * outcome0 * e0)
        - np.mean((1 - z) * y)
    )
    ## IPW estimators
    w1 = pscore + (1 - pscore) / e1
    w0 = pscore * e0 + (1 - pscore)
    ace_ipw0 = np.mean(z * y * w1 / pscore) - np.mean((1 - z) * y * w0 / (1 - pscore))
    ace_ipw = np.mean(z * y * w1 / pscore) / np.mean(z / pscore) - np.mean(
        (1 - z) * y * w0 / (1 - pscore)
    ) / np.mean((1 - z) / (1 - pscore))
    ## doubly robust estimator
    aug = outcome1 / pscore / e1 + outcome0 * e0 / (1 - pscore)
    ace_dr = ace_ipw0 - np.mean((z - pscore) * aug)

    return np.array([ace_reg, ace_ipw0, ace_ipw, ace_dr])

In [3]:
nhanes_bmi = pd.read_csv("nhanes_bmi.csv").iloc[:, 1:]
z, y, x = (
    nhanes_bmi.School_meal.values,
    nhanes_bmi.BMI.values,
    nhanes_bmi.iloc[:, 2:].values,
)
x = (x - x.mean(0)) / x.std(0)

In [4]:
E1 = np.array([1 / 2, 1 / 1.7, 1 / 1.5, 1 / 1.3, 1, 1.3, 1.5, 1.7, 2])
E0 = E1.copy()
est = np.zeros((len(E1), len(E0)))

for i in range(len(E1)):
    for j in range(len(E0)):
        est[i, j] = OS_est(z, y, x, e1=E1[i], e0=E0[j])[3]

In [5]:
pd.DataFrame(est, columns=E0, index=E1).round(2)

,0.500000,0.588235,0.666667,0.769231,1.000000,1.300000,1.500000,1.700000,2.000000
0.500000,14.62,13.62,12.73,11.57,8.96,5.57,3.30,1.04,-2.36
0.588235,11.93,10.93,10.04,8.88,6.27,2.87,0.61,-1.66,-5.05
0.666667,10.13,9.13,8.24,7.08,4.47,1.08,-1.19,-3.45,-6.85
0.769231,8.33,7.33,6.45,5.29,2.67,-0.72,-2.98,-5.25,-8.64
1.000000,5.64,4.64,3.75,2.59,-0.02,-3.41,-5.68,-7.94,-11.34
1.300000,3.57,2.57,1.68,0.52,-2.09,-5.49,-7.75,-10.01,-13.41
1.500000,2.65,1.65,0.76,-0.40,-3.01,-6.41,-8.67,-10.93,-14.33
1.700000,1.94,0.94,0.06,-1.11,-3.72,-7.11,-9.38,-11.64,-15.03
2.000000,1.15,0.15,-0.74,-1.90,-4.51,-7.90,-10.17,-12.43,-15.83
